In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def load_contracts(url, years, extra):
    """
    Fetches the Basketball‑Reference contracts table at `url`,
    cleans it, and computes Salary Count & Average Salary.
    
    years: list of the six season‑column names, e.g.
           ["2022-23","2023-24","2024-25","2025-26","2026-27","2027-28"]
    """
    # Step 1–3: get & parse
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table", {"id": "player-contracts"})
    
    # Step 4: read into pandas
    df = pd.read_html(str(table))[0]
    
    # Step 5: set cols
    if extra:
        df.columns = ["Rk", "Player", "Tm"] + years + ["BIRD"] + ["Guaranteed"]
    else:
        df.columns = ["Rk", "Player", "Tm"] + years + ["Guaranteed"]
    # Step 6: drop repeats/blanks
    df = df[
        df["Player"].notna()
        & (df["Player"] != "Player")
        & (df["Player"] != "Missing value")
    ].copy()
    
    # Step 7: count non-null salaries
    df['Salary Count'] = (
        df.iloc[:, 3:3+len(years)]
          .notnull()
          .sum(axis=1)
          .astype(int)
    )
    
    # Step 8: clean & convert Guaranteed to float
    df['Guaranteed'] = (
        df['Guaranteed']
          .astype(str)
          .str.replace(r'[^0-9\.]', '', regex=True)
    )
    df['Guaranteed'] = pd.to_numeric(df['Guaranteed'], errors='coerce')
    
    # report any failures
    bad = df['Guaranteed'].isna()
    if bad.any():
        print()
        #print("Rows with non‑numeric Guaranteed:",
        #      df.loc[bad, ['Player','Guaranteed']])
    
    # Step 9: compute average
    df['Average Salary'] = df['Guaranteed'] / df['Salary Count']

    
    return df

# --- now just call it for each snapshot:
years21 = ["2020-21","2021-22","2022-23","2023-24","2024-25","2025-26"] #
url21   = "https://web.archive.org/web/20220127184320/https://www.basketball-reference.com//contracts/players.html"
contract21 = load_contracts(url21, years21, True)

years22 = ["2021-22","2022-23","2023-24","2024-25","2025-26","2026-27"]
url22   = "https://web.archive.org/web/20220127184320/https://www.basketball-reference.com//contracts/players.html"
contract22 = load_contracts(url22, years22, True)

years23 = ["2022-23","2023-24","2024-25","2025-26","2026-27","2027-28"]
url23   = "https://web.archive.org/web/20230516051257/https://www.basketball-reference.com/contracts/players.html"
contract23 = load_contracts(url23, years23, False)

years24 = ["2023-24","2024-25","2025-26","2026-27","2027-28","2028-29"]
url24   = "https://web.archive.org/web/20240603223914/https://www.basketball-reference.com/contracts/players.html"
contract24 = load_contracts(url24, years24, False)

years25 = ["2024-25","2025-26","2026-27","2027-28","2028-29","2029-30"]
url25 = "https://www.basketball-reference.com/contracts/players.html"
contract25 = load_contracts(url25, years25, False)

C:\Users\girth\AppData\Local\Temp\ipykernel_13232\2085239294.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


C:\Users\girth\AppData\Local\Temp\ipykernel_13232\2085239294.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


C:\Users\girth\AppData\Local\Temp\ipykernel_13232\2085239294.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\girth\AppData\Local\Temp\ipykernel_13232\2085239294.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


C:\Users\girth\AppData\Local\Temp\ipykernel_13232\2085239294.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [5]:
def load_stats(season):
    """
    Load stats from text{season}.txt and assign column names.
    season: int or str, e.g. 21, '22', 25
    """
    fn = f'text{season}.txt'
    df = pd.read_csv(
        fn,
        sep=',',
        header=None,
        encoding='utf-8'
    )
    df.columns = [
        'Rank','Player','Age','Team','Position',
        'Games','Games Started','MP','FG','FGA','FG%',
        '3P','3PA','3P%','2P','2PA','2P%','eFG%',
        'FT','FTA','FT%','ORB','DRB','TRB',
        'AST','STL','BLK','TOV','PF','PTS',
        'Player-additional'
    ]
    return df

# Now load each season’s stats in one line:
stats21 = load_stats(21)
stats22 = load_stats(22)
stats23 = load_stats(23)
stats24 = load_stats(24)
stats25 = load_stats(25)

In [6]:
def merge_contract_stats(contract_df, stats_df):
    """
    Strips whitespace from both DataFrames' Player columns
    and returns their inner merge on Player.
    """
    # work on copies so we don’t clobber originals
    c = contract_df.copy()
    s = stats_df  .copy()
    
    c['Player'] = c['Player'].str.strip()
    s['Player'] = s['Player'].str.strip()
    
    merged = pd.merge(
        c, s,
        on='Player',
        how='inner',
        suffixes=('_contract','_stats')
    )
    return merged

In [7]:
merged25 = merge_contract_stats(contract25, stats25)
merged24 = merge_contract_stats(contract24, stats24)
merged23 = merge_contract_stats(contract23, stats23)
merged22 = merge_contract_stats(contract22, stats22)
merged21 = merge_contract_stats(contract21, stats21)


In [9]:
cap2021 = 109140000 
cap2022 = 112414000
cap2023 = 123655000 
cap2024 = 136021000
cap2025 = 140588000 

In [10]:
def percent_of_cap(years, df, cap):
    """
    For each column in `years`, removes any commas (or $),
    converts to float, and then adds a
      'Percent of Cap {year}'
    column (in percent).
    """
    df = df.copy()
    
    for year in years:
        # 1) remove commas or dollar signs, then cast to float
        df[year] = (
            df[year]
              .astype(str)
              .str.replace(r'[,\$]', '', regex=True)
              .astype(float)
        )
        
        # 2) compute percent of cap
        df[f'Percent of Cap {year}'] = (df[year] / cap)
    
    return df


In [19]:
final25 = percent_of_cap(years25, merged25, cap2025)
final24 = percent_of_cap(years24, merged24, cap2024)
final23 = percent_of_cap(years23, merged23, cap2023)
final22 = percent_of_cap(years22, merged22, cap2022)
final21 = percent_of_cap(years21, merged21, cap2021)

In [20]:
final22.drop('BIRD', axis=1, inplace=True)
final21.drop('BIRD', axis=1, inplace=True)

In [24]:
def avg_percent_of_cap(years, df):
    """
    Calculates the average of the 'Percent of Cap {year}' columns for each player.
    Ignores missing values during the calculation.
    
    years: list of years for which 'Percent of Cap {year}' columns exist.
    df: DataFrame containing the 'Percent of Cap {year}' columns.
    
    Returns:
        DataFrame with an additional column 'Average Percent of Cap'.
    """
    percent_columns = [f'Percent of Cap {year}' for year in years]
    df['Average Percent of Cap'] = df[percent_columns].mean(axis=1, skipna=True)
    return df

In [25]:
final25 = avg_percent_of_cap(years25, final25)
final24 = avg_percent_of_cap(years24, final24)
final23 = avg_percent_of_cap(years23, final23)
final22 = avg_percent_of_cap(years22, final22)
final21 = avg_percent_of_cap(years21, final21)

In [ ]:
season_dfs = {
    '2020-21': final21,
    '2021-22': final22,
    '2022-23': final23,
    '2023-24': final24,
    '2024-25': final25
}

all_seasons = pd.concat(
    (df.assign(Season=season) for season, df in season_dfs.items()),
    ignore_index=True
)

# 3) (Optional) Reorder so Season is first
cols = ['Season'] + [c for c in all_seasons.columns if c != 'Season']
all_seasons = all_seasons[cols]


(2478, 58)


,Season,Rk,Player,Tm,2020-21,2021-22,2022-23,2023-24,2024-25,2025-26,...,Percent of Cap 2025-26,Average Percent of Cap,2026-27,Percent of Cap 2026-27,2027-28,Percent of Cap 2027-28,2028-29,Percent of Cap 2028-29,2029-30,Percent of Cap 2029-30
0,2020-21,1,Stephen Curry,GSW,45780966.0,48070014.0,51915615.0,55761216.0,59606817.0,NaN,...,NaN,0.478531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-21,2,John Wall,HOU,44310840.0,47366760.0,NaN,NaN,NaN,NaN,...,NaN,0.420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-21,3,Russell Westbrook,LAL,44211146.0,47063478.0,NaN,NaN,NaN,NaN,...,NaN,0.418154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-21,4,James Harden,BRK,43848000.0,46872000.0,NaN,NaN,NaN,NaN,...,NaN,0.415613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-21,5,LeBron James,LAL,41180544.0,44474988.0,NaN,NaN,NaN,NaN,...,NaN,0.392411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
print(all_seasons.columns)

Index(['Season', 'Rk', 'Player', 'Tm', '2020-21', '2021-22', '2022-23',
       '2023-24', '2024-25', '2025-26', 'Guaranteed', 'Salary Count',
       'Average Salary', 'Rank', 'Age', 'Team', 'Position', 'Games',
       'Games Started', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Player-additional',
       'Percent of Cap 2020-21', 'Percent of Cap 2021-22',
       'Percent of Cap 2022-23', 'Percent of Cap 2023-24',
       'Percent of Cap 2024-25', 'Percent of Cap 2025-26',
       'Average Percent of Cap', '2026-27', 'Percent of Cap 2026-27',
       '2027-28', 'Percent of Cap 2027-28', '2028-29',
       'Percent of Cap 2028-29', '2029-30', 'Percent of Cap 2029-30'],
      dtype='object')
